In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
%matplotlib inline
import seaborn as sns

In [13]:
date = '2020-01-12'
path = '../data/{}.csv'.format(date)

In [14]:
df = pd.read_csv(path)
df.time = pd.to_datetime(df.time)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694393 entries, 0 to 694392
Data columns (total 4 columns):
id_module    694393 non-null int64
id_status    694393 non-null int64
value        694393 non-null object
time         694393 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 21.2+ MB


### Lista parametrow.

In [15]:
parameters = {'TC_COMPRESSOR_STATE': 1007, 
              'TC_COMPRESSED_GAS_TEMP': 1022,
              'TC_EVD_VALVE_OPENING': 1674,
              'TC_EVD_CONDENSING_PRESSURE': 1668,
              'TC_COMPRESSOR_LAST_WORK_TIME': 1816,}

### Lista modułów.

In [16]:
modules = df.id_module.unique()
modules

array([20752, 21834, 23082,  5599, 10865, 16031])

### Lista parametrów.

In [17]:
parameters_value = parameters.values()
parameters_value

dict_values([1007, 1022, 1674, 1668, 1816])

### Dane.

In [31]:
def get_value(val):
    arr_val = val.split(',')
    type = int(arr_val[2])
    if type == 1:
        return int(arr_val[6])
    elif type == 6:
        return int(arr_val[11])
    elif type == 11:
        return int(arr_val[4])
    else:
        return np.nan

In [38]:
data = df.copy()
data = data[data.id_status.isin(parameters_value)]
data = data.sort_values('time')
data['val'] = data['value'].apply(get_value)
data = data.drop('value', axis=1)
data.time = data.time.apply(lambda x: x.time().replace(second=0, microsecond=0))

In [33]:
def prepare_data_for_module(id_module, data, parameters):
    data = data[data.id_module == id_module]
    columns = list(parameters.keys())
    columns.append('TIME')
    module_data = pd.DataFrame(columns=columns)
    
    
    module_data['TIME'] = data.time.unique()
    for k in parameters:
        module_data[k] = data[data.id_status == parameters[k]]['val'].values
    return module_data

In [34]:
# fix dla modulu 23082

data_module = prepare_data_for_module(5599, data, parameters)
data_module['WORK_TIMER'] = data_module.TC_COMPRESSOR_STATE * data_module.TC_COMPRESSOR_LAST_WORK_TIME

In [35]:
columns = list(parameters.keys())
columns.append('WORK_TIMER')
corr = data_module[columns].corr()
corr.style.background_gradient(cmap='coolwarm')

,TC_COMPRESSOR_STATE,TC_COMPRESSED_GAS_TEMP,TC_EVD_VALVE_OPENING,TC_EVD_CONDENSING_PRESSURE,TC_COMPRESSOR_LAST_WORK_TIME,WORK_TIMER
TC_COMPRESSOR_STATE,1,0.798962,0.872739,0.61512,-0.31133,0.680182
TC_COMPRESSED_GAS_TEMP,0.798962,1,0.589343,0.849245,-0.0882586,0.687383
TC_EVD_VALVE_OPENING,0.872739,0.589343,1,0.343544,-0.30847,0.548027
TC_EVD_CONDENSING_PRESSURE,0.61512,0.849245,0.343544,1,-0.0781121,0.480802
TC_COMPRESSOR_LAST_WORK_TIME,-0.31133,-0.0882586,-0.30847,-0.0781121,1,0.281475
WORK_TIMER,0.680182,0.687383,0.548027,0.480802,0.281475,1


In [44]:
time_list = pd.to_timedelta(np.arange(24*60), unit='m')
time_list

TimedeltaIndex(['00:00:00', '00:01:00', '00:02:00', '00:03:00', '00:04:00',
                '00:05:00', '00:06:00', '00:07:00', '00:08:00', '00:09:00',
                ...
                '23:50:00', '23:51:00', '23:52:00', '23:53:00', '23:54:00',
                '23:55:00', '23:56:00', '23:57:00', '23:58:00', '23:59:00'],
               dtype='timedelta64[ns]', length=1440, freq=None)